<a href="https://colab.research.google.com/github/ta269uec/semantic-search/blob/develop/k_means_sentence_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing all the packages necessary + loading the data

In [11]:
!pip install datasets transformers torch sentence-transformers --quiet

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
LIB_LOCATION = "/content/drive/MyDrive/git-repos/semantic-search/"
OUT_DATA_LOCATION = LIB_LOCATION + "data/"

In [4]:
# Clone if you do not have this.
#!git clone -b develop https://github.com/ta269uec/semantic-search.git /content/drive/MyDrive/git-repos/semantic-search/

In [14]:
import sys
sys.path.append(LIB_LOCATION)

In [15]:
from src.sentence_encoder import SentenceEncoder
from src.sst2 import get_reviews_sst2
from src.kmeans import cluster_k_means

In [16]:
pos_sentences, neg_sentences = get_reviews_sst2()

In [7]:
st = SentenceEncoder()

In [17]:
pos_embeddings, neg_embeddings = st.encode(pos_sentences), st.encode(neg_sentences)

In [26]:
import os
import pickle
file_name_pos = os.path.join(OUT_DATA_LOCATION, 'sst2_train_pos_embeddings.pkl')
file_name_neg = os.path.join(OUT_DATA_LOCATION, 'sst2_train_neg_embeddings.pkl')
with open(file_name_pos, "wb") as fd:
  pickle.dump(pos_embeddings, fd)
with open(file_name_neg, "wb") as fd:
  pickle.dump(neg_embeddings, fd)

# Build K-Means Clusters

In [18]:
def build_clusters(N, pos_sentences, pos_embeddings, neg_sentences, neg_embeddings):
  clustered_pos_sentences, clustered_pos_embeddings = cluster_k_means(corpus = pos_sentences,\
                                                            corpus_embeddings = pos_embeddings.cpu(),\
                                                            num_clusters = N)
  clustered_neg_sentences, clustered_neg_embeddings = cluster_k_means(corpus = neg_sentences,\
                                                            corpus_embeddings = neg_embeddings.cpu(),\
                                                            num_clusters = N)
  return (clustered_pos_sentences, clustered_pos_embeddings), (clustered_neg_sentences, clustered_neg_embeddings)


In [24]:
import os
import pickle
def serialize_clusters(root, ds_name, label, n_clusters, results):
  file_prefix = f"{ds_name}_{label}_{n_clusters}_"
  sentences, embeddings = results[0], results[1]
  f1, f2 = os.path.join(root, f"{file_prefix}sentences.pkl"), os.path.join(root, f"{file_prefix}embeddings.pkl")
  with open(f1, 'wb') as handle:
    pickle.dump(sentences, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open(f2, 'wb') as handle:
    pickle.dump(sentences, handle, protocol=pickle.HIGHEST_PROTOCOL)    
  return

In [25]:
root = OUT_DATA_LOCATION
for N in [10, 25, 50, 100, 250]:
  print(f"Processing cluster size {N}")
  res_pos, res_neg = build_clusters(N, pos_sentences, pos_embeddings, neg_sentences, neg_embeddings)
  serialize_clusters(root, ds_name, label="pos", n_clusters=N, results=res_pos)
  serialize_clusters(root, ds_name, label="neg", n_clusters=N, results=res_neg)
  pass

Processing cluster size 10


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Processing cluster size 25


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Processing cluster size 50


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Processing cluster size 100


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Processing cluster size 250


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## Sanity Check 1: Are clusters semantically similar?

In [52]:
clusters = [len(clustered_pos_embeddings[i]) for i in range(len(clustered_pos_embeddings))]

In [53]:
import torch
X = torch.stack(clustered_pos_embeddings[0])

In [54]:
from sentence_transformers import util

cos_scores = util.cos_sim(X, X).numpy()

In [55]:
cos_scores

array([[1.        , 0.56275344, 0.16633949, ..., 0.34173852, 0.33133203,
        0.33750123],
       [0.56275344, 1.0000001 , 0.25965416, ..., 0.31524807, 0.38929945,
        0.3176574 ],
       [0.16633949, 0.25965416, 0.9999997 , ..., 0.18074846, 0.2621564 ,
        0.18604575],
       ...,
       [0.34173855, 0.315248  , 0.18074849, ..., 1.0000002 , 0.377027  ,
        0.32030904],
       [0.33133203, 0.38929948, 0.26215637, ..., 0.37702698, 1.        ,
        0.29153726],
       [0.33750117, 0.31765735, 0.18604569, ..., 0.32030913, 0.29153726,
        0.99999976]], dtype=float32)